## Use the CRaftID environment described in sequence analysis section##

Each command will take just a few seconds to run on the single image file provided.

**Scripts required:** 

    stage_position_from_olympus.py
    make_processing_macro_olympus.py
    
**imports**

    import make_processing_macro_olympus as mpo
    import stage_position_from_olympus as spo
    import detect_squares_v2 as grid
    
    import glob
    import os

# (pre-processing) Generate Maximum Projected Tif files from .oir Files Outputted from Microscope

The example provided with this code will take .oir files from the Olympus FV3000 microscope with 3 color channels (red, blue, and brightfield) and 5 z-planes. Slight modifications may be required with other setups. 

**Inputs:**

    1) input_data: Directory where .oir files exist
    
    2) tif_files: Directory where .tif files will be saved after running the macro in Fiji (outside this notebook)
    
    3) common_name_prefix: common prefix in name of all .oir files from the experiment
    
    4) file_list: output from mpo.get_file_list(num_files)
    
    5) macro_file_dir: Folder where macro will be saved
    
**Outputs:**

    Will generate a file called oir_macro.ijm in the assigned folder that can be run in Fiji to process all oir images into a max-projection image for each color channel saved as .tif files in the assigned save_directory.
    
    Must run this file in Fiji with "Plugins" - "Macros" - "Run" to process the images.

**Example command:**

In [1]:
import make_processing_macro_olympus as mpo
import glob
import os

input_data = "/Users/apple/Yi Lab/2021-11-08/oir-tif test/fos-mch-38/" 
tif_files = "/Users/apple/Yi Lab/2021-11-08/oir-tif test/tif/"
macro_file_dir = "/Users/apple/Yi Lab/2021-11-08/oir-tif test/"

if not os.path.exists(tif_files):
    ! mkdir $tif_files
if not os.path.exists(macro_file_dir):
    ! mkdir $macro_file_dir

num_files = len(glob.glob(input_data+"*.oir"))
file_list = ['0001', '0002', '0003', '0004']

common_name_prefix = ["DG green baseline_A01_G001_","DG red baseline_A01_G001_"]

mpo.make_macro_3_channels(input_data, tif_files, common_name_prefix, file_list, macro_file_dir)

['0001', '0002', '0003', '0004']

## (pre-processing) Run the file ```oir_macro.ijm``` in Fiji with "Plugins" - "Macros" - "Run" to process the images.

## (optional) Get absolute x and y coordinates for each image from metadata.##

This will be used later on to select rafts out of the array.

**Inputs:**

    1) Metadatafile from imaging session "matl.omp2info"
    
    2) Filname and path to save result.
    
**Outputs:**

    csv file with 5 columns: filename, x-step, y-step, absolute x-coordinate, absolute y-coordinate. 

**Example Command:**

In [ ]:
import stage_position_from_olympus as spo

metadata = input_data+"matl.omp2info"
savefile = macro_file_dir+"pos_info.csv"
results = spo.master(metadata, filename_to_save=savefile)

## 3) Segment tif images for individual rafts and select rafts containing cells##

**Inputs:**

    1) tif_files: Directory where tif files are stored, one per color channel. 
        (Current naming suffixes: Brightfield = *2.tif, Red Channel = *1.tif, Blue channel = *0.tif)
        
    2) Array type - this is used to specify the length (in pixels) of squares to be detected. 
    Need to adjust with different arrays/microscope settings (Need to make a demo on how to get these numbers). 
    
    3) candidate_wells_dir: Directory where individual raft images will be stored.
    
    4) Blur used to improve square detection in brightfield image. Recommended to set to True
    
**Outputs:**

    All rafts detected with cells will be saved in the assigned folder. Two images per raft, one for red and one for blue channel

**Example Command:**

In [3]:
import detect_squares_v2 as grid

array_type = "custom_100_old_10x_confocal"
tif_files = "tif_files/"
candidate_wells_dir = "candidate_wells/"

if not os.path.exists(candidate_wells_dir):
    ! mkdir $candidate_wells_dir

grid.process_all(tif_files, array_type, candidate_wells_dir, blur=True)

100%|██████████| 324/324 [21:54<00:00,  3.91s/it]